In [10]:
import collections
import enum
import json
import os
import random

In [168]:
import nltk.tokenize

import numpy as np
import pandas as pd

In [12]:
import import_ipynb
import aux.defs
import aux.relation_extraction
import aux.utils
import aux.nlp
import preparation

%run explanation_04.ipynb

In [47]:
STATEMENTS_DIR = "/Users/YK/mt/project/statements_3/"
RACE_PART = "train/middle"
RACE_DIR = "/Users/YK/mt/RACE"
PARSED_RACE_DIR = "/Users/YK/mt/parsed/race"

In [119]:
class Position(enum.Enum):
    BEFORE = "before"
    NESTED = "nested"
    AFTER  = "after"

    
Alternative = collections.namedtuple(
    "Alternative",
    [
        "true_statement", 
        "alternative_statement",
        "relation_type",
        "position", 
        "distance_words",
        "distance_sentences",
        "sn_length",
        "sn_length_relative_difference",
        "jaccard_distance",
        "edit_distance"
    ]
)

In [161]:
def load_statements(directory, subdirectories, text_no):
    statements = {}
    for subdirectory in subdirectories:
        file_path = os.path.join(
            directory, subdirectory, RACE_PART, f"{text_no}.txt.tree"
        )
        if os.path.exists(file_path): 
            with open(file_path, "rt") as f:
                statements[subdirectory] = json.load(f)
    return statements
    

def load_relations(text_no, directory):
    file_path = os.path.join(directory, f"{text_no}.txt.tree")
    if os.path.exists(file_path):
        text, relations, _ = aux.relation_extraction.load_relations(
            os.path.join(directory, f"{text_no}.txt.tree")
        )
        return text, relations # {t: relations[t] for t in types if t in relations}
    else:
        return "", {}

In [154]:
statements_subdirectories = [
    f for f in os.listdir(STATEMENTS_DIR) if os.path.isdir(os.path.join(STATEMENTS_DIR, f))
]

In [162]:
def get_n_words(text_span):
    return len(nltk.tokenize.word_tokenize(text_span))


def get_n_sentences(text_span):
    cnt = 0
    for c in text_span:
        if c in {'.', ',', '!'}:
            cnt += 1
    return cnt


def get_position_and_distance(statement, relation, text, verbose=False):
    if relation.right.end <= statement["left_boundary"]:
        span = text[relation.right.end:statement["left_boundary"]]
        return (
            Position.BEFORE,
            get_n_words(span),
            get_n_sentences(span)
        )
    elif relation.left.start >= statement["right_boundary"]:
        span = text[statement["right_boundary"]:relation.left.start]
        return (
            Position.AFTER,
            get_n_words(span),
            get_n_sentences(span)
        )
    else:
        if (
            relation.left.start < statement["split_point"]
                and relation.right.end > statement["split_point"]
        ):
            if verbose:
                print("The relation overlaps with the relation of the true statement.")
            return None, None, None
        else:
            if relation.right.end <= statement["split_point"]:
                span = text[relation.right.end:statement["split_point"]]
                return Position.NESTED, get_n_words(span), get_n_sentences(span)
            else:
                span = (text[statement["split_point"]:relation.left.start])
                return Position.NESTED, get_n_words(span), get_n_sentences(span)

    
def get_jaccard_distance(phrase_1, phrase_2):
    tokens_1 = set(nltk.tokenize.word_tokenize(phrase_1))
    tokens_2 = set(nltk.tokenize.word_tokenize(phrase_2))
    return nltk.jaccard_distance(tokens_1, tokens_2)


def get_edit_distance(phrase_1, phrase_2):
    return nltk.edit_distance(phrase_1, phrase_2)
    

RelationData = collections.namedtuple(
    "RelationData",
    ["relation", "position", "distance_words", "distance_sentences"]
)


def get_k(relation_data_list, closest, k):
    sorted_relation_data_list = sorted(
        relation_data_list, key=lambda rd: rd.distance_words
    )
    if closest:
        return sorted_relation_data_list[:k]
    else:
        return sorted_relation_data_list[-k:]
        
        
def filter_relations(statement, relations, text, k=2):
    relation_data_lists = collections.defaultdict(list)
    for relation in relations:
        position, distance_words, distance_sentences = get_position_and_distance(
            statement, relation, text
        )
        if position is not None:
            relation_data_lists[position].append(
                RelationData(
                    relation=relation,
                    position=position,
                    distance_words=distance_words,
                    distance_sentences=distance_sentences
                )
            )
    
    result = []
    result += get_k(relation_data_lists[Position.BEFORE], closest=True, k=k)
    result += get_k(relation_data_lists[Position.AFTER], closest=True, k=k)
    result += get_k(relation_data_lists[Position.NESTED], closest=False, k=k)
    return result
    
    
def create_alternative(statement, relation_data, true_sn_text_len, text, verbose=False):
    relation_info = preparation.get_info(relation_data.relation, verbose)
    assert relation_info is not None
    if relation_info.satellite_info.relation is None:
        if verbose:
            print("Satellite is flat.")
        return None

    satellite_handling_result = preparation.Preprocessor.handle_satellite(
        text, relation_info.satellite_info, relation_info.nucleus_info.direction, verbose
    )
    if satellite_handling_result is None:
        if verbose:
            print("Satellite preprocessing was unsuccessful.")
        return None
    
    processed_sn_text = aux.nlp.take_first_sentence_and_remove_leading_words(
        satellite_handling_result.preparation_result.prepared_text, verbose
    )
    prepared_sn_text = utils.lowercase_first_letter(
        processed_sn_text if processed_sn_text is not None 
            else info.satellite_preparation_result.prepared_text
    )
    sn_text_len = get_n_words(prepared_sn_text)
    
    true_statement_nucleus = statement["nucleus"]
    connective = statement["connective"]
    alternative_text = f"{true_statement_nucleus}{connective}{prepared_sn_text}"
    return Alternative(
        true_statement=statement["statement_text"],
        alternative_statement=alternative_text,
        position=relation_data.position.value,
        relation_type=relation_data.relation.type,
        distance_words=relation_data.distance_words,
        distance_sentences=relation_data.distance_sentences,
        sn_length=sn_text_len,
        sn_length_relative_difference=(sn_text_len / true_sn_text_len - 1),
        jaccard_distance=get_jaccard_distance(
            statement["satellite_nucleus"], prepared_sn_text
        ),
        edit_distance=get_edit_distance(
            statement["satellite_nucleus"], prepared_sn_text
        )
    )

In [163]:
def generate_alternatives(text_no):
    rows = []
    
    text, relation_map = load_relations(
        text_no, os.path.join(PARSED_RACE_DIR, RACE_PART)
    )
    if len(relation_map) > 0:
        relations = [
            relation for _, relations in relation_map.items() for relation in relations
        ]
        statement_map = load_statements(STATEMENTS_DIR, statements_subdirectories, text_no)
        for _, statements in statement_map.items():
            for statement in statements:
                true_sn_text_len = get_n_words(statement["satellite_nucleus"])
                filtered_relation_data = filter_relations(statement, relations, text)
                for relation_data in filtered_relation_data:
                    alternative = create_alternative(
                        statement, relation_data, true_sn_text_len, text
                    )
                    if alternative is not None:
                        row_dict = alternative._asdict()
                        row_dict.update(
                            {
                                "text_no": text_no,
                                "rule": statement["rule"],
                                "reason": statement["reason"][1]
                            }
                        )
                        rows.append(
                            row_dict
                        )
    return rows


def create_df(rows):
    if len(rows) > 0:
        result_df = pd.DataFrame(rows)[
            [
                "text_no",
                "true_statement", 
                "alternative_statement",
                "relation_type",
                "position",        
                "distance_words",
                "distance_sentences",
                "sn_length",
                "sn_length_relative_difference",
                "jaccard_distance",
                "edit_distance",
                "rule",
                "reason"
            ]
        ]
        result_df["d"] = (
            result_df.distance_words 
                * (1 - 2 * (result_df.position == Position.NESTED).astype(int))
        )
        result_df.sort_values(
            ["text_no", "rule", "true_statement", "position", "d"], inplace=True
        )
        result_df.drop("d", 1, inplace=True)

        return result_df
    else:
        return None

In [144]:
text_numbers = [
    int(fn.split('.')[0]) for fn in os.listdir(os.path.join(RACE_DIR, RACE_PART))
        if fn[-4:] == ".txt"
]

In [164]:
# rows = []

for i, text_no in enumerate(text_numbers):
    if i < 90:
        continue
    print(f"[{i + 1:4d}/{len(text_numbers)}, text no. {text_no:4d}]")
    rows.extend(generate_alternatives(text_no))

result_df = create_df(rows)

[  91/6409, text no. 5786]
[  92/6409, text no. 7191]
[  93/6409, text no. 5976]
[  94/6409, text no. 7807]
[  95/6409, text no. 7813]
[  96/6409, text no. 3254]
[  97/6409, text no. 7032]
[  98/6409, text no. 1443]
[  99/6409, text no. 7754]
[ 100/6409, text no. 5143]
[ 101/6409, text no. 4249]
[ 102/6409, text no. 3526]
[ 103/6409, text no. 5157]
[ 104/6409, text no. 1331]
[ 105/6409, text no. 7026]
[ 106/6409, text no. 1457]
[ 107/6409, text no. 5631]
[ 108/6409, text no. 3240]
[ 109/6409, text no. 6310]
[ 110/6409, text no. 5619]
[ 111/6409, text no. 2176]
[ 112/6409, text no. 3268]
[ 113/6409, text no. 4507]
[ 114/6409, text no. 4261]
[ 115/6409, text no. 2610]
[ 116/6409, text no. 6476]
[ 117/6409, text no.  855]
[ 118/6409, text no.  699]
[ 119/6409, text no.  841]
[ 120/6409, text no. 4275]
[ 121/6409, text no. 2604]
[ 122/6409, text no. 2162]
[ 123/6409, text no. 4513]
[ 124/6409, text no. 6304]
[ 125/6409, text no. 1480]
[ 126/6409, text no.  114]
[ 127/6409, text no. 2189]
[

[ 396/6409, text no. 5236]
[ 397/6409, text no. 1050]
[ 398/6409, text no. 7421]
[ 399/6409, text no. 2571]
[ 400/6409, text no. 6717]
[ 401/6409, text no. 7409]
[ 402/6409, text no. 6071]
[ 403/6409, text no. 5578]
[ 404/6409, text no. 2217]
[ 405/6409, text no. 4666]
[ 406/6409, text no. 2203]
[ 407/6409, text no. 4672]
[ 408/6409, text no. 8048]
[ 409/6409, text no. 4114]
[ 410/6409, text no. 3690]
[ 411/6409, text no. 1087]
[ 412/6409, text no.  513]
[ 413/6409, text no. 1939]
[ 414/6409, text no. 4699]
[ 415/6409, text no. 4841]
[ 416/6409, text no. 5593]
[ 417/6409, text no. 4855]
[ 418/6409, text no.  261]
[ 419/6409, text no. 7384]
[ 420/6409, text no. 6924]
[ 421/6409, text no.  507]
[ 422/6409, text no. 1093]
[ 423/6409, text no. 3874]
[ 424/6409, text no.  249]
[ 425/6409, text no. 1905]
[ 426/6409, text no. 1911]
[ 427/6409, text no. 4869]
[ 428/6409, text no. 3860]
[ 429/6409, text no. 6918]
[ 430/6409, text no. 1520]
[ 431/6409, text no. 8262]
[ 432/6409, text no. 7151]
[

[ 700/6409, text no. 5948]
[ 701/6409, text no. 5790]
[ 702/6409, text no. 6299]
[ 703/6409, text no. 7187]
[ 704/6409, text no. 5784]
[ 705/6409, text no. 3493]
[ 706/6409, text no. 1284]
[ 707/6409, text no. 6500]
[ 708/6409, text no.  923]
[ 709/6409, text no. 4317]
[ 710/6409, text no. 3478]
[ 711/6409, text no. 2000]
[ 712/6409, text no. 4471]
[ 713/6409, text no. 7178]
[ 714/6409, text no. 1509]
[ 715/6409, text no. 6266]
[ 716/6409, text no. 6272]
[ 717/6409, text no. 4465]
[ 718/6409, text no. 2772]
[ 719/6409, text no. 6514]
[ 720/6409, text no.  937]
[ 721/6409, text no. 2982]
[ 722/6409, text no. 3444]
[ 723/6409, text no. 5035]
[ 724/6409, text no. 1253]
[ 725/6409, text no. 7144]
[ 726/6409, text no. 8277]
[ 727/6409, text no. 1535]
[ 728/6409, text no. 5753]
[ 729/6409, text no. 2028]
[ 730/6409, text no. 3336]
[ 731/6409, text no. 4459]
[ 732/6409, text no. 7150]
[ 733/6409, text no. 1521]
[ 734/6409, text no. 6528]
[ 735/6409, text no. 1247]
[ 736/6409, text no. 7636]
[

[1004/6409, text no. 3938]
[1005/6409, text no. 3910]
[1006/6409, text no. 4919]
[1007/6409, text no. 1861]
[1008/6409, text no.  339]
[1009/6409, text no. 1120]
[1010/6409, text no. 7551]
[1011/6409, text no. 3737]
[1012/6409, text no. 4058]
[1013/6409, text no. 5346]
[1014/6409, text no. 2429]
[1015/6409, text no. 5420]
[1016/6409, text no. 3051]
[1017/6409, text no. 8104]
[1018/6409, text no. 7237]
[1019/6409, text no. 6129]
[1020/6409, text no. 8110]
[1021/6409, text no. 7223]
[1022/6409, text no. 1652]
[1023/6409, text no. 5434]
[1024/6409, text no. 3045]
[1025/6409, text no. 3723]
[1026/6409, text no. 5352]
[1027/6409, text no. 1134]
[1028/6409, text no. 7545]
[1029/6409, text no. 6883]
[1030/6409, text no. 4064]
[1031/6409, text no. 2415]
[1032/6409, text no.  488]
[1033/6409, text no. 2373]
[1034/6409, text no. 4702]
[1035/6409, text no. 2367]
[1036/6409, text no. 4716]
[1037/6409, text no. 3079]
[1038/6409, text no. 6101]
[1039/6409, text no. 1108]
[1040/6409, text no. 7579]
[

[1309/6409, text no. 5620]
[1310/6409, text no. 3251]
[1311/6409, text no. 7037]
[1312/6409, text no. 6329]
[1313/6409, text no. 1446]
[1314/6409, text no. 1452]
[1315/6409, text no. 5634]
[1316/6409, text no. 3245]
[1317/6409, text no. 5152]
[1318/6409, text no. 1334]
[1319/6409, text no.   38]
[1320/6409, text no. 7745]
[1321/6409, text no.  878]
[1322/6409, text no. 7976]
[1323/6409, text no. 5807]
[1324/6409, text no. 5813]
[1325/6409, text no.  887]
[1326/6409, text no. 7962]
[1327/6409, text no. 5185]
[1328/6409, text no. 2832]
[1329/6409, text no. 7792]
[1330/6409, text no. 3292]
[1331/6409, text no. 3286]
[1332/6409, text no. 2198]
[1333/6409, text no.  105]
[1334/6409, text no. 1491]
[1335/6409, text no. 7786]
[1336/6409, text no.  663]
[1337/6409, text no. 6498]
[1338/6409, text no. 5191]
[1339/6409, text no. 2826]
[1340/6409, text no. 6117]
[1341/6409, text no. 6671]
[1342/6409, text no. 2417]
[1343/6409, text no. 5378]
[1344/6409, text no. 4066]
[1345/6409, text no. 3709]
[

...

[4973/6409, text no. 7211]
[4974/6409, text no. 1674]
[4975/6409, text no. 7205]
[4976/6409, text no. 3063]
[4977/6409, text no. 5412]
[4978/6409, text no. 5374]
[4979/6409, text no. 3705]
[4980/6409, text no.  486]
[4981/6409, text no. 2433]
[4982/6409, text no. 4042]
[4983/6409, text no. 6655]
[4984/6409, text no. 6133]
[4985/6409, text no. 1884]
[4986/6409, text no. 4724]
[4987/6409, text no. 2355]
[4988/6409, text no. 4730]
[4989/6409, text no. 2341]
[4990/6409, text no. 6127]
[4991/6409, text no. 7239]
[4992/6409, text no. 1890]
[4993/6409, text no. 6899]
[4994/6409, text no. 6641]
[4995/6409, text no. 5348]
[4996/6409, text no. 3739]
[4997/6409, text no. 4056]
[4998/6409, text no.  451]
[4999/6409, text no.  337]
[5000/6409, text no. 4903]
[5001/6409, text no.  323]
[5002/6409, text no.  445]
[5003/6409, text no. 6696]
[5004/6409, text no. 7588]
[5005/6409, text no. 4081]
[5006/6409, text no. 3936]
[5007/6409, text no. 2396]
[5008/6409, text no. 1847]
[5009/6409, text no. 2382]
[

[5278/6409, text no. 6520]
[5279/6409, text no. 5029]
[5280/6409, text no. 2746]
[5281/6409, text no. 4337]
[5282/6409, text no. 4451]
[5283/6409, text no. 6246]
[5284/6409, text no. 1529]
[5285/6409, text no. 7158]
[5286/6409, text no. 2975]
[5287/6409, text no. 7819]
[5288/6409, text no. 2961]
[5289/6409, text no. 5968]
[5290/6409, text no. 5798]
[5291/6409, text no. 4486]
[5292/6409, text no. 5940]
[5293/6409, text no. 6291]
[5294/6409, text no. 7831]
[5295/6409, text no. 1298]
[5296/6409, text no. 2949]
[5297/6409, text no. 2791]
[5298/6409, text no. 2785]
[5299/6409, text no. 7825]
[5300/6409, text no.  718]
[5301/6409, text no. 6285]
[5302/6409, text no. 5613]
[5303/6409, text no. 3262]
[5304/6409, text no. 7004]
[5305/6409, text no. 1475]
[5306/6409, text no. 1313]
[5307/6409, text no.  687]
[5308/6409, text no. 7762]
[5309/6409, text no. 3504]
[5310/6409, text no. 3510]
[5311/6409, text no. 5161]
[5312/6409, text no. 6468]
[5313/6409, text no.  693]
[5314/6409, text no. 1307]
[

[5582/6409, text no. 2545]
[5583/6409, text no. 4134]
[5584/6409, text no. 3897]
[5585/6409, text no. 2551]
[5586/6409, text no. 4120]
[5587/6409, text no. 1058]
[5588/6409, text no. 6737]
[5589/6409, text no. 6051]
[5590/6409, text no. 3129]
[5591/6409, text no. 2237]
[5592/6409, text no. 6079]
[5593/6409, text no.  282]
[5594/6409, text no. 1716]
[5595/6409, text no. 8054]
[5596/6409, text no. 7367]
[5597/6409, text no. 3101]
[5598/6409, text no. 5570]
[5599/6409, text no. 5216]
[5600/6409, text no. 2579]
[5601/6409, text no. 7415]
[5602/6409, text no. 1064]
[5603/6409, text no. 5202]
[5604/6409, text no. 3115]
[5605/6409, text no. 5564]
[5606/6409, text no. 1702]
[5607/6409, text no.  296]
[5608/6409, text no. 7373]
[5609/6409, text no. 2233]
[5610/6409, text no. 4642]
[5611/6409, text no. 8078]
[5612/6409, text no. 6055]
[5613/6409, text no. 6733]
[5614/6409, text no. 4124]
[5615/6409, text no. 3893]
[5616/6409, text no. 2555]
[5617/6409, text no. 4130]
[5618/6409, text no. 2541]
[

[5886/6409, text no. 5830]
[5887/6409, text no. 3299]
[5888/6409, text no. 3272]
[5889/6409, text no. 5603]
[5890/6409, text no. 7014]
[5891/6409, text no. 7772]
[5892/6409, text no. 1303]
[5893/6409, text no.  697]
[5894/6409, text no. 5165]
[5895/6409, text no. 5171]
[5896/6409, text no. 3500]
[5897/6409, text no. 7766]
[5898/6409, text no. 6478]
[5899/6409, text no. 1317]
[5900/6409, text no. 1471]
[5901/6409, text no. 7000]
[5902/6409, text no. 4509]
[5903/6409, text no. 5617]
[5904/6409, text no. 2178]
[5905/6409, text no. 1459]
[5906/6409, text no. 6336]
[5907/6409, text no. 7028]
[5908/6409, text no. 4521]
[5909/6409, text no. 2150]
[5910/6409, text no. 2636]
[5911/6409, text no. 5159]
[5912/6409, text no. 4247]
[5913/6409, text no. 3528]
[5914/6409, text no.  873]
[5915/6409, text no. 6450]
[5916/6409, text no.   33]
[5917/6409, text no. 7996]
[5918/6409, text no.   27]
[5919/6409, text no. 6444]
[5920/6409, text no. 7982]
[5921/6409, text no. 2622]
[5922/6409, text no. 2144]
[

[6197/6409, text no. 3066]
[6198/6409, text no. 4709]
[6199/6409, text no. 5417]
[6200/6409, text no. 2378]
[6201/6409, text no. 1842]
[6202/6409, text no. 6693]
[6203/6409, text no.  468]
[6204/6409, text no. 3933]
[6205/6409, text no. 4090]
[6206/6409, text no. 3927]
[6207/6409, text no. 6687]
[6208/6409, text no. 1856]
[6209/6409, text no. 3099]
[6210/6409, text no.  332]
[6211/6409, text no.  454]
[6212/6409, text no. 6877]
[6213/6409, text no.  440]
[6214/6409, text no. 4912]
[6215/6409, text no.  326]
[6216/6409, text no. 6862]
[6217/6409, text no.  441]
[6218/6409, text no.  327]
[6219/6409, text no. 4913]
[6220/6409, text no. 4907]
[6221/6409, text no.  333]
[6222/6409, text no. 6876]
[6223/6409, text no. 6686]
[6224/6409, text no. 3926]
[6225/6409, text no. 4091]
[6226/6409, text no. 3098]
[6227/6409, text no. 2386]
[6228/6409, text no. 1857]
[6229/6409, text no. 1843]
[6230/6409, text no. 2392]
[6231/6409, text no. 3932]
[6232/6409, text no.  469]
[6233/6409, text no. 6692]
[

In [165]:
result_df.head()

,text_no,true_statement,alternative_statement,relation_type,position,distance_words,distance_sentences,sn_length,sn_length_relative_difference,jaccard_distance,edit_distance,rule,reason
12392,2,Drink the water that has not been boiled becau...,Drink the water that has not been boiled becau...,Evaluation,before,0,0,15,0.071429,0.826087,57,explanation_01,Satellite's nucleus contains 'because'.
12390,2,Drink the water that has not been boiled. That...,Drink the water that has not been boiled. That...,Evaluation,before,0,0,15,0.071429,0.826087,57,explanation_02,Common pattern; nucleus has nested relations.
12391,2,Drink the water that has not been boiled. More...,Drink the water that has not been boiled. More...,Evaluation,before,0,0,15,0.071429,0.826087,57,explanation_03,Satellite's nucleus contains 'in fact' / 'as a...
11786,3,There is not enough oil in the world now. That...,There is not enough oil in the world now. That...,Elaboration,nested,0,0,4,-0.826087,0.800000,75,explanation_02,Common pattern; nucleus without nested relations.
11787,3,There is not enough oil in the world now. In f...,There is not enough oil in the world now. In f...,Elaboration,nested,0,0,4,-0.826087,0.800000,75,explanation_04,Common pattern ( -Elaboration).


In [166]:
result_df.to_excel(
    os.path.join(
        STATEMENTS_DIR, 
        f"alternatives_{RACE_PART.replace('/', '-')}_{random.randint(0, 2**32):x}.xlsx"
    ),
    index=False
)

In [180]:
result_df.columns

Index(['text_no', 'true_statement', 'alternative_statement', 'relation_type',
       'position', 'distance_words', 'distance_sentences', 'sn_length',
       'sn_length_relative_difference', 'jaccard_distance', 'edit_distance',
       'rule', 'reason'],
      dtype='object')

In [205]:
sum(
    (result_df.jaccard_distance >= 0.3)
        & (np.abs(result_df.sn_length_relative_difference) <= 0.5)
        & ((result_df.position != "nested") | (result_df.distance_words > 0))
)

10586

In [206]:
sum(
    (result_df.jaccard_distance >= 0.0)
        & (np.abs(result_df.sn_length_relative_difference) <= 0.5)
        & ((result_df.position != "nested") | (result_df.distance_words > 0))
)

10597

In [200]:
reduced_df = result_df.loc[
    (result_df.sn_length <= 20)
        & (result_df.jaccard_distance >= 0.3)
        & (np.abs(result_df.sn_length_relative_difference) <= 0.5)
        & ((result_df.position != "nested") | (result_df.distance_words > 0))
]

selected_true_statements = np.random.choice(
    list(set(reduced_df.true_statement)), 200, replace=False
)

reduced_df = reduced_df.loc[
    reduced_df.true_statement.isin(set(selected_true_statements))
]

In [201]:
len(reduced_df) / len(result_df)

0.013650232856913441

In [202]:
len(reduced_df)

255

In [203]:
reduced_df.to_excel(
    os.path.join(
        STATEMENTS_DIR, 
        f"prefiltered_alternatives_{RACE_PART.replace('/', '-')}_{random.randint(0, 2**32):x}.xlsx"
    ),
    index=False
)